In [0]:
dbutils.fs.ls("abfss://raw@1formula1dl11.dfs.core.windows.net")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
cuircuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                      StructField("circuitRef", StringType(), True),
                                      StructField("name", StringType(), True),
                                      StructField("location", StringType(), True),
                                      StructField("country", StringType(), True),
                                      StructField("lat", DoubleType(), True),
                                      StructField("lng", DoubleType(), True),
                                      StructField("alt", IntegerType(), True),
                                      StructField("url", StringType(), True)])
cuircuits_df = spark.read \
.schema(cuircuits_schema ) \
.option("header", "true") \
.csv("abfss://raw@1formula1dl11.dfs.core.windows.net/circuits.csv")

cuircuits_df.show(10)
cuircuits_df.printSchema()

In [0]:
from pyspark.sql.functions import col, current_timestamp

column_select_df = cuircuits_df.select(col("name"), col("circuitId"), col("circuitRef"), col("location"), col("country"), col("lat"), col("lng"), col("alt"))

column_rename_df = column_select_df.withColumnRenamed("circuitId", "circuit_id") \
.withColumnRenamed("circuitRef", "circuit_ref") \
.withColumnRenamed("lat", "latitude") \
.withColumnRenamed("lng", "longitude") \
.withColumnRenamed("alt", "altitude") \
.withColumn("ingestion_date", current_timestamp())


column_rename_df.show(10)

In [0]:
column_rename_df.write.mode("overwrite").parquet("abfss://processed@1formula1dl11.dfs.core.windows.net/circuits")
display(dbutils.fs.ls("abfss://processed@1formula1dl11.dfs.core.windows.net/circuits"))